In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends
import pickle

import pyExSi

import LadiskDAQ

%matplotlib qt

In [2]:
acq1 = LadiskDAQ.NIAcquisition("TestInputTask", acquisition_name="NI_task")

acq2 = LadiskDAQ.SerialAcquisition(port="COM3", baudrate=250000, timeout=5,
                                  byte_sequence=(("uint32", 1), ("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb",
                                  end_bytes=b"\n",
                                  pretest_time= 60.,
                                  sample_rate=500.,
                                  acquisition_name="Arduino_1"
                                  )
#acq2.set_trigger(1, 0, duration=2) #TODO

acq3 = LadiskDAQ.SerialAcquisition(port="COM8", baudrate=250000, timeout=5,
                                  byte_sequence=(("uint32", 1), ("int16", 1), ("int16", 1)),
                                  start_bytes=b"\xfa\xfb", 
                                  end_bytes=b"\n",
                                  pretest_time= None,
                                  sample_rate=200,
                                  acquisition_name="Arduino_2"
                                  )

In [3]:
acq1.sample_rate, acq2.sample_rate, acq3.sample_rate

(25600.0, 500.0, 200)

In [4]:
# create DAQ object:
ldaq = LadiskDAQ.Core(acquisitions=[acq1, acq2, acq3])

In [5]:
ldaq.run(run_time=5.0)

+--------+--------------------------------------------------+
| HOTKEY |                   DESCRIPTION                    |
+--------+--------------------------------------------------+
|   q    |               Stop the measurement               |
+--------+--------------------------------------------------+
|   s    | Start the measurement manually (without trigger) |
+--------+--------------------------------------------------+
|   f    |      Freeze the plot during the measurement      |
+--------+--------------------------------------------------+
| Space  |           Resume the plot after freeze           |
+--------+--------------------------------------------------+
	Waiting for trigger...triggered.
	Recording...

In [11]:
fig, axs = plt.subplots(3, sharex=True)
axs[0].plot( ldaq.acquisitions[0].time, ldaq.acquisitions[0].data )
axs[1].plot( ldaq.acquisitions[1].time, ldaq.acquisitions[1].data[:, 1:] )
axs[2].plot( ldaq.acquisitions[2].time, ldaq.acquisitions[2].data[:, 1:] )

In [7]:
ldaq.acquisitions[1].data[:, 0], ldaq.acquisitions[2].data[:, 0] 

(array([2170024., 2172024., 2174024., ..., 7180024., 7182024., 7184024.]),
 array([1500024., 1502024., 1504024., ..., 6512024., 6514024., 6516024.]))

In [13]:
plt.figure()
plt.plot( ldaq.acquisitions[1].plot_time, ldaq.acquisitions[1].plot_data ) 

In [29]:
i = 2

data = ldaq.acquisitions[i].data
time_arr = np.arange(len(data))/ldaq.acquisitions[i].sample_rate
time_sent = data[:, 0]

In [30]:
#plt.figure()
#plt.plot(np.diff(time_sent*1e-3))

( time_arr[-1]-time_arr[0] - 1e-6*(time_sent[-1]-time_sent[0]) )*1e3


#60 seconds: 56.40274111675181 ms

335.51058823529445